In [20]:
import torchcraft
from torchcraft import replayer
import os
import pickle
import numpy as np
import data_utils
from generate_role_datasets import unit_to_dict, game_over_time, post_process, hmm_data


In [21]:
replays_master = open('good_files.txt', 'r').readlines()
for i in range(len(replays_master)):
    replays_master[i] = replays_master[i].split('\n')[0]

In [22]:
num_games_to_parse = 1
step_size = 3
window_size = 20
# I only want information on these units:
terran_valid_types = [0, 1, 2, 3, 5, 7, 8, 9, 11, 12, 13, 30, 32, 34, 58]
zerg_valid_types = [37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 50, 62, 103]
protoss_valid_types = [60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 83, 84, 85]
zerg_babies = [35, 36, 59, 97]
valid_units = terran_valid_types  # + zerg_valid_types + protoss_valid_types



In [ ]:
# Cell for data generation across many games
all_games = []
for replay_path in replays_master:
    if len(all_games) >= num_games_to_parse:
        break
    game_info = game_over_time(replay_path, valid_types=valid_units, playerid=2, step_size=step_size)
    if game_info != -1:
        all_games.append(game_info)
        processed = post_process(game_info, n_timesteps=window_size)
        print("Length of game: %s" % len(game_info))
        print("Number of Processed: %s" % len(hmm_samples))
        X = []
        for i in range(0, len(processed), 20):
            X.append(processed[i].values())


In [23]:
# Cell for data generation across 1 game (debugging)
replay_path = '/media/asilva/HD_home/StarData/dumped_replays/0/bwrep_poa7y.tcr'
game_info = game_over_time(replay_path, valid_types=valid_units, playerid=0, step_size=3)
processed = post_process(game_info, n_timesteps=window_size)
X = []
for i in range(0, len(processed), 20):
    X.append(processed[i].values())

loaded replay: /media/asilva/HD_home/StarData/dumped_replays/0/bwrep_poa7y.tcr of length:4546


In [24]:
print len(processed)
print len(game_info)
print len(X)
print len(X[0])
print X[0]

1520
1516
76
4
[[0.0, 0.03521046167445302, 0.6, 0.0, 1.0], [0.0, 0.03756910949545184, 0.6, 0.0, 1.0], [0.0, 0.006176323555016366, 0.6, 0.0, 1.0], [0.0, 0.0390625, 0.6, 0.0, 1.0]]


In [8]:
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.externals import joblib

num_clusters = 5

two_means = cluster.MiniBatchKMeans(n_clusters=num_clusters)

gmm = mixture.GaussianMixture(
    n_components=num_clusters,
    covariance_type='full')



In [9]:
clustering_algorithms = (
    ('MiniBatchKMeans', two_means),
    ('GaussianMixture', gmm)
)

In [10]:
for name, algorithm in clustering_algorithms:
    algorithm.fit(X)
    joblib.dump(algorithm, 'clf/'+name+'.pkl') 